In [49]:
import numpy as np
import pandas as pd
import pymongo

In [50]:
myclient = pymongo.MongoClient("mongodb://119.3.185.83:27017/")
db = myclient["bibtex_train"]
db_cache = myclient["cache_bibtex_train"]

In [45]:
def dict_to_list(DICT):
    dictlist = []
    for key,value in DICT.items():
        dictlist.append({"key": key, "count": value})
    return dictlist

In [47]:
features = db["features"].find({})
z = []
for doc in features:
    z.append(doc)
df = pd.DataFrame(z)
df = df.set_index("_id")
db_cache["none"].insert_many(dict_to_list(dict(df.sum())))

In [52]:
fils = db["graph_filters"]
for fil in fils.find({}):
    query_docs = db["labels"].find(fil["query"])
    query_result = [query_doc["_id"] for query_doc in query_docs]
    docs=db["features"].find({"_id": {"$in": query_result}})
    z = []
    for doc in docs:
        z.append(doc)
    df = pd.DataFrame(z)
    df = df.set_index("_id")
    db_cache[str(fil["_id"])].insert_many(dict_to_list(dict(df.sum())))